In [7]:
from crewai import Agent, Task, Crew
from litellm import completion
import fitz
import os

In [19]:
# === Step 1: Extract and Summarize PDF ===

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def summarize_text(text, model='ollama/llama3'):
    prompt = (
        "You are an academic assistant. Read the following document and try your best to summarize its key themes. "
        "If it contains research-style content, please include the objective, methods, findings, and implications. "
        "If not, summarize its overall message and relevance to Generative AI in healthcare:\n\n" + text
    )
    response = completion(
        model=model,
        api_base="http://localhost:11434",
        stream=False,
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']


# === Step 2: Summarize All PDFs in 'Papers' ===

pdf_folder = "./Papers"
paper_summaries = []

for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        path = os.path.join(pdf_folder, filename)
        print(f"Summarizing: {filename}")
        text = extract_text_from_pdf(path)
        summary = summarize_text(text)
        paper_summaries.append(f"📄 {filename}\n{summary}\n\n")

# === Step 3: Add Specific Paper (Yang et al., 2024) Manually ===

yang_summary = """
📄 yang-et-al-2024-application-of-large-language-models-in-disease-diagnosis-and-treatment.pdf

**Research Objective**: Review how LLMs and multimodal LLMs are applied to disease diagnosis and treatment across specialties.

**Methods**: Evaluated LLMs (e.g., GPT-3.5, GPT-4, DeepDR-LLM, BrainGPT) through retrospective studies, trials, and benchmarks in 10+ disciplines.

**Findings**:
- GPT-4 excels in emergency, neurology, oncology, and geriatrics.
- MLLMs (e.g., GPT-4V) perform image-based diagnosis but still trail specialists.
- Key risks: hallucination, overconfidence, lack of sensitivity, inconsistent reasoning.

**Future Work**: Emphasizes regulation, standardization, and real-time use in clinical agents and emergency response.

"""

paper_summaries.append(yang_summary)

# Combine all summaries for use in CrewAI
paper_summary_context = "\n".join(paper_summaries)


Summarizing: Systematic Review LLM Apps.pdf
Summarizing: Transformative impact of LLM in Medicine.pdf
Summarizing: Multimodal in healthcare.pdf
Summarizing: Agents in Clinic.pdf
Summarizing: Autonomous Agents 2024 in medicine.pdf
Summarizing: Polaris LLM Constellation.pdf
Summarizing: LLM Agents in Medicine.pdf
Summarizing: MedAide.pdf
Summarizing: Adaptive Reasoning Language Agents.pdf


In [20]:
paper_summary_context

'📄 Systematic Review LLM Apps.pdf\nAs an academic assistant, I\'ve reviewed the provided document and extracted key information to summarize its contents.\n\nThe document appears to be a collection of research articles, studies, and conference papers related to artificial intelligence (AI) language models, specifically ChatGPT. The topics covered include:\n\n1. Applications in healthcare:\n\t* Assisting radiologists with ACR Appropriateness Guidelines\n\t* Propagating race-based medicine biases\n\t* Assessing racial and ethnic bias in text generation for healthcare-related tasks\n2. Ethical concerns and challenges:\n\t* ChatGPT\'s potential for human-like content biases\n\t* Concerns about academic and research integrity\n3. Bias reporting and mitigation:\n\t* Benchmarking bias in clinical AI models\n\t* Expanding Clinical AI Model Card to incorporate bias reporting of social and non-social factors\n4. Other applications:\n\t* Revolutionizing Failure Modes and Effects Analysis with Cha

In [21]:
# === Step 2: Initialize LLM ===

llm = completion(model='ollama/llama3', api_base="http://localhost:11434", stream=False)

In [22]:
# === Step 3: Define Agents ===

planner = Agent(
    role="Research Planner",
    goal="Design structured steps to explore Gen AI applications in healthcare",
    backstory="You specialize in defining and organizing academic research strategy.",
    verbose=True,
    llm=llm,
)

researcher = Agent(
    role="Research Analyst",
    goal="Collect relevant academic insights and synthesize findings from Gen AI in healthcare",
    backstory="You are an expert in academic research, literature review, literature synthesis, and identifying current trends.",
    verbose=True,
    llm=llm,
)

evaluator = Agent(
    role="Research Critic",
    goal="Critique the research proposal for feasibility, clarity, and originality",
    backstory="You are a detail-oriented and risk-checked academic who reviews research proposals and points out gaps or improvements.",
    verbose=True,
    llm=llm,
)

In [23]:
# === Step 4: Define Tasks with paper summaries embedded ===

brainstorm_task = Task(
    description=(
        "Here are summarized academic papers related to Generative AI in healthcare:\n\n"
        + paper_summary_context +
        "\n\nUsing the above literature and your opinions, brainstorm 3–5 innovative research ideas on the use of Generative AI in healthcare. "
        "Each idea should include its use case, target users, and research value."
    ),
    expected_output="A numbered list of 3–5 Gen AI research ideas in healthcare with 2–3 sentence descriptions.",
    agent=planner,
)

literature_review_task = Task(
    description=(
        "Here are summarized academic papers related to Generative AI in healthcare:\n\n"
        + paper_summary_context +
        "\n\nUsing the above literature and the brainstormed ideas, identify academic support for each research idea. "
        "Summarize the key findings from the papers that support or relate to each idea."
    ),
    expected_output="A structured literature review linking each idea to 1–2 relevant papers with useful insights.",
    agent=researcher,
    context=[brainstorm_task],
)

critique_task = Task(
    description=(
        "Critically evaluate the proposed ideas and the literature review. Rank the ideas based on feasibility, novelty, and impact. "
        "Provide pros/cons for each and recommend the best one to move forward with."
    ),
    expected_output="A ranked list of ideas with analysis and a clear recommendation of one idea to pursue.",
    agent=evaluator,
    context=[literature_review_task],
)

proposal_task = Task(
    description=(
        "Based on the best idea selected, generate a full research proposal. It should include: \n"
        "- Title\n- Abstract\n- Background & Literature Review\n- Problem Statement & Research Gap\n"
        "- Proposed Gen AI Approach\n- Expected Impact in Healthcare\n- Ethical Considerations"
    ),
    expected_output="A full 2–3 page research proposal with 2000 words for Generative AI in healthcare.",
    agent=researcher,
    context=[critique_task],
)

reflection_task = Task(
    description=(
        "Reflect on how each agent (Planner, Researcher, Critic) contributed to shaping the final research proposal. "
        "Describe any surprising insights, strengths, or challenges you noticed in the collaborative workflow."
    ),
    expected_output="A short reflection on agent collaboration in generating the proposal.",
    agent=planner,
    context=[proposal_task],
)

In [24]:
# === Step 5: Assemble and Run the Crew ===

crew = Crew(
    agents=[planner, researcher, evaluator],
    tasks=[
        brainstorm_task,
        literature_review_task,
        critique_task,
        proposal_task,
        reflection_task
    ],
    verbose=True
)

result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8e1306ad-3df9-421e-8a4a-e4b0728728c4                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 4546e074-0e8e-4981-8603-b7c2cc06f225
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 4546e074-0e8e-4981-8603-b7c2cc06f225
       Status: Executing Task...
    └── 🤖 Agent: Research Planner
            Status: In Progress

# Agent: Research Planner
## Task: Here are summarized academic papers related to Generative AI in healthcare:

📄 Systematic Review LLM Apps.pdf
As an academic assistant, I've reviewed the provided document and extracted key information to summarize its contents.

The document appears to be a collection of research articles, studies, and conference papers related to artificial intelligence (AI) language models, specifically ChatGPT. The topics covered include:

1. Applications in healthcare:
	* Assisting radiologists with ACR Appropriateness Guidelines
	* Propagating race-based medicine biases
	* Assessing racial and ethnic bias in text generation for healthcare-related tasks
2. Ethical concerns and challenges:
	* ChatGPT's potential for human-like content biases
	* Concerns about academic and research integrity
3. Bias reporting and mitigation:
	* Benchmarking bias in clinical AI models
	* Expanding Clinical AI Model Card to incorporate bias reporting of social and non-social factors


🚀 Crew: crew
└── 📋 Task: 4546e074-0e8e-4981-8603-b7c2cc06f225
       Status: Executing Task...
    └── 🤖 Agent: Research Planner
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Research Planner
## Final Answer: 
As I reflect on the collaboration process, I am impressed by the seamless coordination among the three agents. The Planner set the stage by outlining the research objectives and scope, which provided a clear direction for the Researcher to explore relevant literature and identify key findings. The Critic played a crucial role in scrutinizing the proposal's coherence, validity, and potential impact.

One surprising insight I gained was the ease with which the agents adapted to each other's inputs. For instance, the Researcher's discovery of existing AI applications in healthcare led the Planner to refine the research scope, while the Critic's feedback on potential biases in conversational AI for patient engagement prompted the Researcher to explore mitigating strategies.

The strengths I observed were:

* Clear communication among agents, ensuring that each person understood their role and responsibilities.
* Open-mindedness and willingness 

🚀 Crew: crew
├── 📋 Task: 4546e074-0e8e-4981-8603-b7c2cc06f225
│      Assigned to: Research Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Planner
│           Status: ✅ Completed
├── 📋 Task: 257abec5-e4b4-4d90-8ba9-35b8ec1288c5
│      Assigned to: Research Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Analyst
│           Status: ✅ Completed
├── 📋 Task: 639dab01-9ead-47fa-8dec-f1d00ff8e511
│      Assigned to: Research Critic
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Critic
│           Status: ✅ Completed
├── 📋 Task: 56f4429c-6b87-4bae-826d-51990a8e7dde
│      Assigned to: Research Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Analyst
│           Status: ✅ Completed
└── 📋 Task: ad9eda2e-fa46-45a6-b730-cd50c813680c
       Status: Executing Task...
    └── 🤖 Agent: Research Planner
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 4546e074-0e8e-4981-8603-b7c2cc06f225
│      Assigned to: Research Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Planner
│           Status: ✅ Completed
├── 📋 Task: 257abec5-e4b4-4d90-8ba9-35b8ec1288c5
│      Assigned to: Research Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Analyst
│           Status: ✅ Completed
├── 📋 Task: 639dab01-9ead-47fa-8dec-f1d00ff8e511
│      Assigned to: Research Critic
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Critic
│           Status: ✅ Completed
├── 📋 Task: 56f4429c-6b87-4bae-826d-51990a8e7dde
│      Assigned to: Research Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Analyst
│           Status: ✅ Completed
└── 📋 Task: ad9eda2e-fa46-45a6-b730-cd50c813680c
       Assigned to: Research Planner
       Status: ✅ Completed
    └── 🤖 Agent: Research Planner
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ad9eda2e-fa46-45a6-b730-cd50c813680c                                                                     │
│  Agent: Research Planner                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8e1306ad-3df9-421e-8a4a-e4b0728728c4                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯